## Why Do We Need Vectors? 

"If vectors are *basically* like lists, except that they can only store data of one type, why on earth do we need them?" I hear you asking.

Speed and memory usage.

As we mentioned in our last reading, lists are flexible, but that flexibility comes at the cost of performance and memory usage. In other words, if lists and cars were vehicles, lists would be 18 wheelers -- lots of space to store stuff, but big and hard to get around -- and numpy vectors would be sports cars -- small and fast, but not suited for carrying just anything around.

How much slower? Well, to illustrate let's create a list with all the numbers from one to a billion and sum them with regular Python; then let's do the same thing with numpy vectors and compare the result:

In [ ]:
import numpy as np

# Remember `range` doesn't include the last number,
# so I have to go up to 1_000_000_001 to actually get all
# the numbers from 1 to 1_000_000_000

# Make list
one_to_billion_list = list(range(1, 1_000_000_001))

# May array
one_to_billion_vector = np.arange(1, 1_000_000_001)


In [ ]:
%time sum(one_to_billion_list)

CPU times: user 11.8 s, sys: 35.8 s, total: 47.7 s
Wall time: 1min 5s


500000000500000000

In [ ]:
%time np.sum(one_to_billion_vector)

CPU times: user 1.77 s, sys: 5.95 s, total: 7.72 s
Wall time: 10.1 s


500000000500000000

So on my 2019 Macbook Pro with 32GB RAM, that took about ~1 minute with regular Python; ~10 seconds with numpy. That's more than a 50x speedup, just for that simple calculation.

But that's not all -- to create that list, regular Python required over **35GB** of RAM, while numpy only used about 6GB.

This is, of course, a toy example. But storing large collections of numbers and manipulating them quickly is at the heart of data science, and these differences in speed and storage efficiency are precisely why we use numpy arrays instead of regular Python objects like lists. 

But why does this happen?

## Computer Memory

When most of us think about computer memory, we think of it as an amorphous place where our data just... goes. But the reality is that memory a physical thing in our computer, and a big part of why lists and vectors have such different performance characteristics is down to how they use make use of that physical resource. 

Computer memory can be thought of as a big field of buckets (transistors), each of which is capable of holding an individual 1 or 0, and each of which has an address. Storing something in memory is just the process of(a) picking a block of buckets that aren't currently being used, (b) setting the 1s and 0s in those buckets to the data values you want stored, and (c) then keeping a record of the address where those 1s and 0s have been put.

Accessing something in memory, analogously, consists of giving the computer an address and asking it to go find the buckets specified by that address and read out the 0s and 1s they contain.

(I know, I know -- if you've taken a course on computer architecture, I'm sure this analogy is *deeply* offensive to your deeper understanding of how all this works, but bear with me).

Now, you might *assume* that when you create a list in Python -- say, by typing the code `x = ["a", 42, 3.14]` -- that Python would go find a big unused area of memory and put "the list" (e.g. `["a", 42, 3.14]`) in one place. But that's not actually what happens. 

When you create a list in Python, what Python *actually* does is create every item in the list as a separate object in a distinct location in memory. That means that Python goes out to memory, finds an open block of buckets, and stores, say, `"a"` in one place. Then in another place -- a place picked more or less at random, for reasons that aren't worth getting into -- it finds some buckets to store `42`. And finally, it finds another place to put `3.14`. *Then* it actually creates *another* data structure where it writes down where all those entries have been put, which I'll call the "address book" for the moment. 

So if you ask for the first item in a list (`x[0]`), what Python actually does is looks up the location of the address book, reads out the first address stored in that book, then goes to that address to find the first entry in the list.

Uh, *what* you say?!

Oh, it gets even weirder, because when Python stores `"a"` in memory, it isn't just storing the 0s and 1s `01100001` (how "a" is represented in binary in nearly all string encoding schemes, like unicode). Oh no, that would make *much* too much sense! Instead, it has to store: "Hello! I'm a Python object. Please interpret what follows as a string: `01100001`" so that when Python gets to that object, it knows that `01100001` means "a" and not the integer 97. Similarly, when Python stores 42, it has to store: "Hello! I'm a Python object. Please interpret what follows as an integer: `101010`". And for 3.14, it has to store: "Hello! I'm a Python object. Please interpret what follows as a floating point number: (long series of numbers not worth including)".

This is important because Python needs to know what kind of data is stored in `x[0]` just to know what it's being asked to do. For example, consider the code `x[0] * 2`. This code can actually have to different meanings depending on whether `x[0]` is a string (e.g. `"a"`) or a number (e.g. `97`). If `x[0]` is the string "a", then `x[0] * 2` should return `"aa"`; but if `x[0]` is 97, then `x[0] * 2` should return `194`.

So to reiterate, for Python to run the code `x[0] * 2`, it must:

- Go to the address table for the list `x` and find the first address;
- Go to *that* address and read the data;
- Based on what that data says is its type (here, a string), it can decide what it's being asked to do;
- Then it can ask the computer's processor to create a doubled string (`"aa"`).














In [1]:
"a" * 2

'aa'